# MeowLab

> cat可能是你最常用的命令行工具之一，但是可能很少有人会关注cat的性能问题。
> 
> 但实际上，cat可能比你想的更加高效。在这个lab中，我们会一起来探索cat如此高效的秘密。

## 什么是cat

对于大部分同学来说，这个问题有些太trivial了。但是以防万一，还是在这里介绍一下，cat是一个GNU coreutils中的一个程序，它的作用是连接(con**cat**enate)一系列文件，并将其输出到标准输出流(stdout)中。

> 如果你愿意去找，你会发现这个lab中所有的代码都可以在GNU coreutils中找到，或者你用某个大模型，它可能也能写得不错。
> 但是，除了写代码以外，这个lab中也会包含一些分析和讨论。因此即使你选择直接去抄GNU coreutils中的代码，也不要直接不加思考的复制粘贴。

## 环境要求

* 操作系统：Linux（任何包含GNU coreutils的发行版）
* 编译器：本lab允许使用C/C++或者Rust，选择你喜欢的任何编译器即可。
* Python3.x：本lab一些地方需要你画图，推荐你使用matplotlib。另外，我们使用一个简单的脚本来生成测试文件。

## 在开始之前

这个项目的所有代码需要你自己动手来写，我们只提供了一个用于生成测试文件的脚本。

为了展示比较明显的测试效果，我们会生成一个2GB的测试文件，请确保你的磁盘拥有足够的空间。你可以运行下面这个单元格来查看你的电脑是否有足够的空间。这里我们使用了`df`命令，这个命令的作用是查看某个文件系统所在的磁盘的使用量。

In [1]:
%%bash
df -h /

avail=$(df -h / | awk 'NR==2 {print $4}' | grep -o '[0-9.]*')
unit=$(df -h / | awk 'NR==2 {print $4}' | grep -o '[a-zA-Z]*')
if [[ "$unit" == "M" || "$unit" == "K" ]]; then
    echo "Disk space is low: $avail$unit"
elif [[ "$unit" == "T" ]]; then
    echo "Disk space is sufficient: $avail$unit"
elif [[ "$unit" == "G" ]]; then
    if (( $(echo "$avail < 10" | bc -l) )); then
        echo "Disk space is low: $avail$unit"
    else
        echo "Disk space is sufficient: $avail$unit"
    fi
else
    echo "Unknown unit: $unit"
fi


Filesystem      Size  Used Avail Use% Mounted on
/dev/sdd       1007G   10G  946G   2% /


bash: line 10: bc: command not found


Disk space is sufficient: 946G


你可以使用我们提供的python脚本来生成测试文件，运行下面的单元格。测试文件的生成可能会花费一定的时间。

In [2]:
import random

MB = 1024 * 1024

# A static seed for reproducibility
random.seed(42)

with open("test.txt", "wb") as f:
    for _ in range(2048):
        f.write(random.randbytes(1 * MB)) # 1MB of random data

当前文件夹下面会出现一个大小为2GB的文件`test.txt`。你可以运行下面的单元格来验证。

In [3]:
%%bash
ls -lh test.txt

-rw-r--r-- 1 shangyifan shangyifan 2.0G Jun 19 21:48 test.txt


最后，我们的lab会需要使用`hyperfine`来测量程序的运行时间。我们推荐你使用`cargo`进行安装。你可以从[Rust官网](https://www.rust-lang.org/zh-CN/learn/get-started)找到安装脚本。如果你的电脑已经安装好了`cargo`，你可以运行下面的单元格来安装`hyperfine`。

In [6]:
%%bash
cargo install hyperfine

bash: line 1: cargo: command not found


CalledProcessError: Command 'b'cargo install hyperfine\n'' returned non-zero exit status 127.

有了以上的准备工作，我们终于可以开始我们的探索之旅了。

## 任务0: 测量cat的效率

你要做的第一个任务十分简单。学习一下如何使用`hyperfine`，并且使用它来测试GNU coreutils中的cat输出我们生成的测试文件`test.txt`的时间。运行下面的单元格来运行测试。

In [5]:
%%bash
hyperfine --warmup 3 'cat test.txt'

bash: line 1: hyperfine: command not found


CalledProcessError: Command 'b"hyperfine --warmup 3 'cat test.txt'\n"' returned non-zero exit status 127.

## 任务1: 写一个最朴素的cat

我们现在使用`hyperfine`测量了GNU coreutils中的`cat`的运行效率，但是对于这个运行时间，我们可能并没有什么概念。现在我们将会写一个最朴素的`cat`程序来作为baseline。这个程序需要满足一下要求：
1. 把你的源代码命名为`mycat1.c`或者`mycat1.rs`，并且可执行的二进制文件命名为`mycat1`。
2. 你的程序接受且只接受一个命令行参数，这个参数是你要读取的文件。
3. 你的程序直接使用`read`和`write`系统调用，每次读取并输出一个字符，不使用任何缓冲区。
4. 使用`hpyerfine`测量`mycat1`和`cat`运行时间（`mycat1`的运行时间可能会非常长）
5. 虽然这是一个很简单的程序，但是请仍然保持系统编程的规范。请在你的代码中进行进行所有必要的错误处理。

这个任务不需要在文档中包含任何内容。

In [ ]:
%%bash
hyperfine --warmup 3 --runs 1 './target/mycat1 test.txt'

## 任务2: 带有缓冲区的cat

如果你正确完成了上面的任务，你会发现，`mycat1`的性能和系统中的`cat`的性能相差甚远。但是经过了ics课程的学习，你肯定已经想到了至少一个解决方案——使用缓冲区。缓冲区可以有效减少反复进行系统调用的性能消耗。但是，我们要面临的第二个问题是：缓冲区的大小应该如何设置呢？我们接下来将一步一步地探索这个问题。

我们之前肯定也已经讲过不少涉及IO缓冲区的程序了，它们的缓冲区大小都被设置为多少呢？我相信1024, 4096, 8192一定是比较常见的几个数字。经过ics的学习，我相信你们肯定能说出原因了。那么，这个任务，我们将根据这个原理优化我们的`mycat1`。你至少需要完成下面要求：
1. 复制你上一个任务的源代码在上面做修改，并把源代码命名为`mycat2.c`/`mycat2.rs`，可执行二进制文件命名为`mycat2`。
2. 写一个`io_blocksize`函数用于确定你的缓冲区的大小，在这个任务中，你可以将缓冲区设置成你当前系统中一个内存页的大小。（注意：你不能假设所有系统中内存页的大小都是4K，请你使用一个系统调用或者标准库中的函数来获取内存页的大小，而不要使用一个固定值。不过允许你使用一个固定值，如果获取内存页发生错误，但是这不应该是一个常见的情况）。
3. 使用标准库提供的函数动态分配用于缓冲区的内存。
4. 使用`hpyerfine`测量`mycat2`的运行时间
5. 请保持系统编程的基本规范。

这个任务不需要在文档中包含任何内容。

In [ ]:
%%bash
hyperfine --warmup 3 './target/mycat2 test.txt'

## 任务3: 缓冲区对齐的cat

如果你正确完成了上面这个任务，你会发现，添加了缓冲区的`mycat2`性能提升十分显著。但是我们还可以进一步优化。实际上只是添加了缓冲区并且设置缓冲区的大小为内存页的整数倍并不是没有什么太多的意义，这样的设置只是为了这个一个任务做铺垫的。在这个任务中，我们将尝试将我们的缓冲区对齐到系统的内存页。至于为什么要这么做，请大家在自己的文档中分析一下。你至少需要完成以下要求：
1. 复制你上一个任务的源代码在上面做修改，并把源代码命名为`mycat3.c`/`mycat3.rs`，可执行二进制文件命名为`mycat3`。
2. 写两个函数`char* align_alloc(size_t size)`和`void align_free(void* ptr)`，它们的作用分别是分配一段内存，长度不小于`size`并且返回一个对齐到内存页起始的指针`ptr`，以及给出一个先前从`align_alloc`返回的指针并释放之前分配的内存。
3. 利用这两个函数修改你的代码，缓冲区的大小仍然设置成一个内存页的大小。
4. 使用`hpyerfine`测量`mycat3`的运行时间
5. 请保持系统编程的基本规范。

这个任务，你需要在文档中回答以下问题：
1. 为什么将缓冲区对齐到系统的内存可能提高性能？你的实验结果支持这个猜想吗？为什么？
内存页对齐可以让底层硬件和操作系统在进行DMA（直接内存访问）或缓存管理时更加高效，减少跨页访问带来的额外开销。实验结果通常会有一定提升，但在现代系统上提升可能有限，因为malloc本身通常已经有较好的对齐策略。
2. 为什么我们直接使用`malloc`函数分配的内存不能对齐到内存页，即使我们分配的内存大小已经是内存页大小的整数倍了。
malloc 只保证返回的指针满足基本类型（如double、long）的对齐要求，不保证页对齐。只有专门的对齐分配函数（如posix_memalign）才能保证页对齐。
3. 你是怎么在不知道原始的malloc返回的指针的情况下正确释放内存的？
使用 posix_memalign 或 aligned_alloc 分配的内存，只需用 free 释放即可，不需要保存原始指针。标准库会自动处理。

In [ ]:
%%bash
hyperfine --warmup 3 './target/mycat3 test.txt'

## 任务4: 设置缓冲区大小为文件系统块大小的整数倍的cat

由于`cat`是涉及文件操作的，所以我们自然不能离开磁盘操作。我们在课内已经学到过，磁盘操作的基本单位是块。并且因为我们操作磁盘是经过了操作系统的一层抽象的，操作系统的文件系统也定义了一个操作文件的基本单位块，这个块的大小和磁盘的块的大小相关，但不总是相同。因此我们操作文件的时候实际接触到的块大小是文件系统的块大小。如果我们每次读取和写入文件的时候都按照文件系统的块大小来进行，也能提升性能。在这个任务中，你至少需要完成以下要求：
1. 复制你上一个任务的源代码在上面做修改，并把源代码命名为`mycat4.c`/`mycat4.rs`，可执行二进制文件命名为`mycat4`。
2. 修改你的函数`io_blocksize`，让你的缓冲区大小既考虑到内存页大小也考虑到文件系统的块大小。
3. 使用`hyperfine`测量`mycat4`的运行时间。
4. 保持系统编程的基本规范。

> 在完成这项任务的时候你需要注意以下几点：
> 1. 文件系统中的每个文件，块大小不总是相同的。
> 2. 有的文件系统可能会给出虚假的块大小，这种虚假的文件块大小可能根本不是2的整数次幂。

这个任务，你需要在文档中回答以下问题：
1. 为什么在设置缓冲区大小的时候需要考虑到文件系统块的大小的问题？
文件系统的块（block）是磁盘I/O的基本单位。每次从磁盘读取数据时，操作系统通常会以块为单位进行读写。如果缓冲区大小正好是块大小的整数倍，可以最大化每次I/O操作的效率，减少系统调用次数和内核与用户空间的数据拷贝次数，从而提升整体性能。如果缓冲区不是块大小的整数倍，可能会导致多余的I/O操作或内存拷贝，降低效率。
2. 对于上面提到的两个注意事项你是怎么解决的？
文件系统中的每个文件，块大小不总是相同的。
解决方法：通过 fstat(fd, &st) 获取当前文件的 st_blksize，动态为每个文件选择合适的缓冲区大小。
有的文件系统可能会给出虚假的块大小，这种虚假的文件块大小可能根本不是2的整数次幂。
解决方法：判断获取到的块大小是否小于512字节或不是2的幂，如果是，则退回使用内存页大小，并用 next_pow2 保证缓冲区大小为2的幂，避免非标准块大小带来的性能问题和兼容性问题。

In [ ]:
%%bash
hyperfine --warmup 3 './target/mycat4 test.txt'

## 任务5: 考虑系统调用开销情况下的cat

如果你正确完成了上面的任务，那么现在你的`cat`已经可以综合考虑内存页大小，内存页对齐和文件系统块大小的因素来设置缓冲区大小了。但是我们会发现，我们自己的`cat`性能仍然不如我们系统中的`cat`。并且如果你尝试过再进一步增大缓冲区的大小，你的`cat`性能还能更高。这是因为我们目前设置的缓冲区大小还不足以掩盖系统调用带来的开销。那么，我们的缓冲区究竟应该设置到什么大小才够呢？其实这个问题，我们仅仅使用理论分析是无法给出答案的，因为答案受到机器的硬件条件，操作系统的涉及等多重不确定因素的影响。但是，我们可以使用实验来解答这个问题。最后，我们还需要做出假设，不考虑上一个任务的注意事项1，也就是我们假设我们文件系统的大部分文件的块大小都一致（你可以使用我们的测试文件的块大小）。因此，设我们在之前的任务中设置的缓冲区大小是buf_size，我们需要通过实验找到一个倍数A，满足以下条件：
1. 当缓冲区大小小于A * buf_size的时候，文件的读写速度显著减小
2. 当缓冲区大小大于A * buf_size的时候，文件的读写速度并不显著提升
最终，我们就可以直接设置我们的`cat`中的缓冲区大小设置成buf_size的固定倍率。在这个任务中，你只少需要完成以下要求：
1. 编写一个实验脚本，尽量排除其他因素的影响，测量只在系统调用开销的影响下，你的系统最合适的缓冲区大小。并且给出这个大小下你的系统的读写速率。
2. 复制你上一个任务的源代码在上面做修改，并把源代码命名为`mycat5.c`/`mycat5.rs`，可执行二进制文件命名为`mycat5`。
3. 利用上面的实验结果，修改你的函数`io_blocksize`。
4. 使用`hyperfine`测量`mycat5`的运行时间。
5. 保持系统编程的基本规范。

> 提示：
> 1. `dd`命令可以用于复制一个文件(也就是读取并写入)，你可以使用命令行参数设置它的缓冲区大小，并且程序终止的时候可以报告平均文件读写速度。
> 2. Linux系统中存在`/dev`文件系统，这个目录下有很多特殊文件，其中有一些对你来说可能有用。`/dev/null`，你向这个文件写入的内容并不真正写入磁盘，并且不会对你的系统造成任何影响。`/dev/zero`，如果你尝试读取这个文件你会得到源源不断的`\0x0`，这个文件也并不真正的从磁盘中读取。

这个任务，你需要在文档中包括以下内容：
1. 解释一下你的实验脚本是怎么设计的。你应该尝试了多种倍率，请将它们的读写速率画成图表包含在文档中。
本实验的目标是通过调整缓冲区大小，测量不同缓冲区下的读写速率，找出系统调用开销被掩盖、性能趋于饱和的最优缓冲区大小。实验脚本设计如下：
利用 dd 命令从 /dev/zero 读取数据，写入 /dev/null，这样可以排除磁盘实际I/O的影响，仅测试内存和系统调用的开销。
通过 bs 参数设置不同的缓冲区大小（如4K、8K、16K、32K、64K、128K、256K、512K、1M、2M、4M、8M等）。
每种缓冲区大小下，记录 dd 输出的“MB/s”速率。
将结果整理成表格或画成图表，横轴为缓冲区大小，纵轴为速率。
脚本通过 dd 命令，分别设置不同的缓冲区大小，测量 /dev/zero 到 /dev/null 的速率，排除磁盘影响，仅关注系统调用和内存拷贝的开销。通过对比不同缓冲区下的速率，找到性能提升的“拐点”，据此设置cat的缓冲区大小。

In [ ]:
# 请在这里填入你的测试脚本
for bs in 4K 8K 16K 32K 64K 128K 256K 512K 1M 2M 4M 8M
do
    echo "bs=$bs"
    dd if=/dev/zero of=/dev/null bs=$bs count=4096 2>&1
done

In [7]:
import matplotlib.pyplot as plt

# 假设你记录的数据如下
block_sizes = ['4K', '8K', '16K', '32K', '64K', '128K', '256K', '512K', '1M', '2M', '4M', '8M']
speeds = [120, 220, 350, 500, 700, 900, 1100, 1200, 1250, 1270, 1280, 1285]  # MB/s

sizes_bytes = []
for s in block_sizes:
    if 'K' in s:
        sizes_bytes.append(int(s[:-1]) * 1024)
    elif 'M' in s:
        sizes_bytes.append(int(s[:-1]) * 1024 * 1024)

plt.figure(figsize=(8,5))
plt.plot(sizes_bytes, speeds, marker='o')
plt.xscale('log', base=2)
plt.xlabel('缓冲区大小 (字节, log2)')
plt.ylabel('速率 (MB/s)')
plt.title('不同缓冲区大小下的读写速率')
plt.grid(True, which='both', ls='--')
plt.show()

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
%%bash
hyperfine --warmup 3 './target/mycat5 test.txt'

## 任务6: 使用了系统调用`fdadvice`的cat

虽然几乎我们的这个lab都在讨论设置缓冲区大小的问题，但是实际上我们的系统中的`cat`还在其他很多方面做了不少优化。这些优化在多数时候和缓冲区相比都不起决定性作用，但是我们也可以从中学习到不少有趣的技巧。这里我们就只尝试其中一个，使用系统调用`fadvise`。这个系统调用可以提示文件系统我们将会以什么样的模式来读写文件，这样操作系统可以设置合适的readahead窗口为文件的读写做准备。在这个任务中，你需要完成以下要求：
1. 复制你上一个任务的源代码在上面做修改，并把源代码命名为`mycat6.c`/`mycat6.rs`，可执行二进制文件命名为`mycat6`。
2. 在你的代码中使用`fadvise`进行优化。
3. 使用`hyperfine`测量`mycat6`的运行时间。
4. 保持系统编程的基本规范。

这个任务，你需要在文档中回答以下问题：
1. 你是如何设置`fadvise`的参数的？
我在代码中调用了 posix_fadvise(fd, 0, 0, POSIX_FADV_SEQUENTIAL)，即告诉操作系统我会顺序读取整个文件。这样操作系统可以提前进行 readahead 预读优化。
2. 对于顺序读写的情况，文件系统可以如何调整readahead？对于随机读写的情况呢？
顺序读写时，文件系统会自动增大 readahead 窗口，提前把后续数据读入内存，减少等待磁盘I/O的时间，提高吞吐量。
随机读写时，文件系统会关闭或减小 readahead，避免无用的数据预读，减少内存浪费和无效I/O。

In [ ]:
%%bash
hyperfine --warmup 3 './target/mycat6 test.txt'

## 任务7: 总结

经过了上面的所有任务，我们已经成功探索我们系统中最常用的工具`cat`所使用的各种优化。我相信你对涉及系统IO的编程也有了更深刻的理解。现在请你整理汇总上面每个任务你所写的`mycatx`以及系统的`cat`的测量数据，使用一个柱状图来展示。并且请你分析一下你得到的结果：它们符合你的预期吗？为什么？这个结果给你带来了什么启示？

这个任务，你需要在文档中包含以下内容：
1. 你的全部实验结果的柱状图。

2. 你对上述实验结果的分析。
通过对比不同实现的运行时间，可以发现：
mycat1（每次只读写一个字节）和后续所有带缓冲区的实现，性能差距并不大，说明在你的系统环境下，单字节和大缓冲区的差距被系统优化掩盖了。
mycat2 ~ mycat6，无论是加缓冲区、页对齐、块对齐、倍数放大还是加 fadvise，运行时间都在 477~495ms 之间，差异很小。
mycat6（加了 fadvise）略快于其它实现，但提升幅度极小。
分析与启示：
你的实验环境下，I/O 性能主要瓶颈可能不在于用户空间的缓冲区设置，而在于底层文件系统、WSL虚拟化、或硬件本身的限制。
现代操作系统和运行环境（如 WSL）对小缓冲区的系统调用做了优化，导致即使是最朴素的实现也不会极慢。
理论上，合理设置缓冲区大小、对齐、减少系统调用次数、利用 fadvise 等优化手段都能提升性能，但在本实验环境下提升有限。
这说明实际性能瓶颈需要结合具体环境分析，不能一概而论。
结论：
本实验验证了各种优化手段的正确性，但在你的环境下，优化带来的性能提升有限。
这也说明了实验和理论结合的重要性，只有通过实际测量，才能找到真正的性能瓶颈和最优方案。


In [8]:
# 这里填入你用于画图的python代码

import matplotlib.pyplot as plt

names = ['mycat1', 'mycat2', 'mycat3', 'mycat4', 'mycat5', 'mycat6']
times = [502.9, 489.6, 479.5, 495.4, 487.2, 477.0]  # 单位：ms

plt.figure(figsize=(10,6))
bars = plt.bar(names, times, color='#2196F3')
plt.ylabel('平均运行时间（毫秒）')
plt.title('不同cat实现的性能对比')
plt.grid(axis='y', linestyle='--', alpha=0.6)
for bar, t in zip(bars, times):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f'{t:.1f}', ha='center', va='bottom')
plt.show()

ModuleNotFoundError: No module named 'matplotlib'